# Create Recommendations:


In [2]:
import pandas as pd
import numpy as np

def generate_user_recommendations(user_df, question_df, user_id_to_analyze, id_column_name='id'):
    """Generates recommendations for a given user based on their performance and question data."""

    # Check for required columns in both DataFrames
    user_required_cols = [id_column_name, 'submitted_at', 'quiz_created_at', 'accuracy', 'quiz_topic', 'quiz_difficulty_level', 'quiz_name']
    question_required_cols = ['question_id', 'question_difficulty_level', 'question_topic', 'question_type']

    user_missing_cols = [col for col in user_required_cols if col not in user_df.columns]
    question_missing_cols = [col for col in question_required_cols if col not in question_df.columns]

    if user_missing_cols:
        return f"Error: Missing columns in user data: {user_missing_cols}"
    if question_missing_cols:
        return f"Error: Missing columns in question data: {question_missing_cols}"

    try:
        user_df[id_column_name] = pd.to_numeric(user_df[id_column_name], errors='coerce')
    except KeyError:
        return f"Error: Column '{id_column_name}' not found."
    except Exception as e:
        return f"Error converting '{id_column_name}' to numeric: {e}"

    user_data = user_df[user_df[id_column_name] == user_id_to_analyze].copy()
    if user_data.empty:
        return f"No data found for ID: {user_id_to_analyze} in column '{id_column_name}'"

    try:
        user_data['submitted_at'] = pd.to_datetime(user_data['submitted_at'])
        user_data['quiz_created_at'] = pd.to_datetime(user_data['quiz_created_at'])
        user_data['time_taken'] = (user_data['submitted_at'] - user_data['quiz_created_at']).dt.total_seconds()

        if user_data['accuracy'].dtype == object:
            user_data['accuracy'] = user_data['accuracy'].str.replace('%', '', regex=False).astype(float) / 100
        elif pd.api.types.is_numeric_dtype(user_data['accuracy']):
            user_data['accuracy'] = pd.to_numeric(user_data['accuracy'], errors='coerce')

        user_data.dropna(subset=['accuracy'], inplace=True)
        if user_data.empty:
            return f"No valid accuracy data found for ID: {user_id_to_analyze} in column '{id_column_name}'"

        # 1. Weak Areas (Quiz Topics)
        topic_performance = user_data.groupby('quiz_topic')['accuracy'].mean().sort_values()
        weakest_topics = topic_performance.head(3).index.tolist() #Get topic names

        # 2. Difficulty Level Recommendations
        average_accuracy = user_data['accuracy'].mean()
        if average_accuracy < 0.5:
            difficulty_recommendation = "Focus on Easy and Medium difficulty questions to build a stronger foundation."
        elif average_accuracy < 0.75:
            difficulty_recommendation = "Continue practicing Medium difficulty questions and gradually introduce Hard questions."
        else:
            difficulty_recommendation = "You're performing well. Continue practicing Hard questions to master the topics."

        # 3. Question Type Recommendations (using question_df)
        topic_question_types = question_df[question_df['question_topic'].isin(weakest_topics)]['question_type'].value_counts()
        recommended_question_types = topic_question_types.index.tolist()[:2] #Recommend top 2 types

        recommendations = [
            f"Focus on the following topics where you need improvement: {', '.join(weakest_topics)}",
            difficulty_recommendation,
            f"Practice the following question types within those topics: {', '.join(recommended_question_types)}"
        ]
        return "\n".join(recommendations)

    except Exception as e:
        return f"Error during analysis: {e}"

def main():
    try:
        user_df = pd.read_csv('api_Endpoin.csv')
        question_df = pd.read_csv('quiz_endpoint.csv')
        id_column_name = 'id'

        while True:
            try:
                user_id_input = input(f"Enter {id_column_name} (or 'exit'): ")
                if user_id_input.lower() == 'exit':
                    break

                try:
                    user_id_to_analyze = int(user_id_input)
                except ValueError:
                    print("Invalid input. Please enter a valid integer ID or 'exit'.")
                    continue

                insights = generate_user_recommendations(user_df, question_df, user_id_to_analyze, id_column_name)
                print(insights)

            except Exception as e:
                print(f"Error during user analysis: {e}")

    except FileNotFoundError as e:
        print(f"Error: File not found: {e.filename}")
    except pd.errors.EmptyDataError as e:
        print(f"Error: Empty data in file: {e}")
    except pd.errors.ParserError as e:
        print(f"Error parsing file: {e}")
    except Exception as e:
        print(f"An unexpected error occurred during file processing: {e}")

if __name__ == "__main__":
    main()

Enter id (or 'exit'):  333242


Focus on the following topics where you need improvement: Body Fluids and Circulation 
You're performing well. Continue practicing Hard questions to master the topics.
Practice the following question types within those topics: 


Enter id (or 'exit'):  exit
